# Description

Ce notebook est utilisé pour requérir la découpe d’un raster d’une zone d’une couche de données de WaPOR en utilisant l’API de WaPOR

Vous aurez besoin d’une clé de l’API de WaPOR pour utiliser ce notebook.

# Étape 1: Lire la clé d’API

Obtenir votre clé de l’API à partir de https://wapor.apps.fao.org/profile

In [1]:
import requests
import pandas as pd

path_query=r'https://io.apps.fao.org/gismgr/api/v1/query/'
path_sign_in=r'https://io.apps.fao.org/gismgr/api/v1/iam/sign-in/'

APIToken=input('Votre clé d’API: ')

Your API token: Enter your API token


# Étape 2: Obtenir une clé d’autorisation d’accès

Utilisation de la clé d’entrée d’API pour obtenir la clé d’autorisation d’accès

In [2]:
resp_signin=requests.post(path_sign_in,headers={'X-GISMGR-API-KEY':APIToken})
resp_signin = resp_signin.json()
AccessToken=resp_signin['response']['accessToken']
AccessToken

'eyJhbGciOiJSUzI1NiIsImtpZCI6IjIxODQ1OWJiYTE2NGJiN2I5MWMzMjhmODkxZjBiNTY1M2UzYjM4YmYiLCJ0eXAiOiJKV1QifQ.eyJuYW1lIjoiQmljaCBUcmFuIiwicGljdHVyZSI6Imh0dHBzOi8vbGg0Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tQkxvTDhOYUVLNm8vQUFBQUFBQUFBQUkvQUFBQUFBQUFCTDgvMlFJTzFRQUlxbW8vcGhvdG8uanBnIiwiaXNzIjoiaHR0cHM6Ly9zZWN1cmV0b2tlbi5nb29nbGUuY29tL3VuZmFvLWFwcHMtZGV2IiwiYXVkIjoidW5mYW8tYXBwcy1kZXYiLCJhdXRoX3RpbWUiOjE1OTQ4MDg3MTIsInVzZXJfaWQiOiJSNk5Ibkd5dDRUT3p1M212T2tteE04UkxFMnUxIiwic3ViIjoiUjZOSG5HeXQ0VE96dTNtdk9rbXhNOFJMRTJ1MSIsImlhdCI6MTU5NDgwODcxMiwiZXhwIjoxNTk0ODEyMzEyLCJlbWFpbCI6ImJpY2h0bjk0QGdtYWlsLmNvbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJmaXJlYmFzZSI6eyJpZGVudGl0aWVzIjp7Imdvb2dsZS5jb20iOlsiMTAyMTY0MjA1NjAzMjU1NDMxMjQzIl0sImVtYWlsIjpbImJpY2h0bjk0QGdtYWlsLmNvbSJdfSwic2lnbl9pbl9wcm92aWRlciI6ImN1c3RvbSJ9fQ.bFB3eiTYgCdIuyZ3zhkP9YuxJ5F54rPPiitC9nPD6mEEbS4affIeNhtHytb_P07w2wLlNBnegETIeNVj9CivDmKfpia8viOhC3sT5STP_inva7t0_RubNQm6GjKkIFC9A35gT9Q9jdaSAWnFgc5QqLsCU9rR2m5b15O4SrIxLDgD0Ps3-OzBlYlscCWppRckteAvqhOWsi1CQEPgk

# Étape 3: Poster la charge de la requête

Pour plus d’exemples de charges de requête de séries chronologique d’une zone visiter https://io.apps.fao.org/gismgr/api/v1/swagger-ui/examples/AreaStatsTimeSeries.txt 

In [3]:
cube_code='L2_PHE_S'
workspace='WAPOR_2'
outputFileName='L2_PHE_17s1_s_clipped.tif'

#obtenir la mesure du cube de données
cube_url=f'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/{workspace}/cubes/{cube_code}/measures'
resp=requests.get(cube_url).json()
measure=resp['response']['items'][0]['code']
print('MEASURE: ',measure)

#Obtenir la dimension du cube de données
cube_url=f'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/{workspace}/cubes/{cube_code}/dimensions'
resp=requests.get(cube_url).json()
items=pd.DataFrame.from_dict(resp['response']['items'])
items

MEASURE:  PHE


,caption,code,hierarchical,index,links,type,workspaceCode
0,Year,YEAR,False,1,"[{'rel': 'self', 'href': 'https://io.apps.fao....",TIME,WAPOR_2
1,Season,SEASON,False,2,"[{'rel': 'self', 'href': 'https://io.apps.fao....",WHAT,WAPOR_2
2,Growing Stage,STAGE,False,3,"[{'rel': 'self', 'href': 'https://io.apps.fao....",WHAT,WAPOR_2


Définir les valeurs des dimensions pour identifier la donnée raster voulue

In [4]:
year="[2017-01-01,2018-01-01)"
stage="SOS"
season="S1"

## Définir la zone par l’étendue de la coordonnée

In [5]:
bbox= [37.95883206252312, 7.89534, 43.32093, 12.3873979377346] #latlon
xmin,ymin,xmax,ymax=bbox[0],bbox[1],bbox[2],bbox[3]
Polygon=[
          [xmin,ymin],
          [xmin,ymax],
          [xmax,ymax],
          [xmax,ymin],
          [xmin,ymin]
        ]

query={
  "type": "CropRaster",
  "params": {
    "properties": {
      "outputFileName": outputFileName,
      "cutline": True,
      "tiled": True,
      "compressed": True,
      "overviews": True
    },
    "cube": {
      "code": cube_code,
      "workspaceCode": workspace,
      "language": "en"
    },
    "dimensions": [
      {
        "code": "SEASON",
        "values": [
          season
        ]
      },
      {
        "code": "STAGE",
        "values": [
          stage
        ]
      },
      {
        "code": "YEAR",
        "values": [
          year
        ]
      }
    ],
    "measures": [
      measure
    ],
    "shape": {
      "type": "Polygon",
      "coordinates": [Polygon]
    }
  }
}

## Ou définir la zone en passant par GeoJSON

In [ ]:
import ogr
shp_fh=r".\data\Awash_shapefile.shp"

shpfile=ogr.Open(shp_fh)
layer=shpfile.GetLayer()
epsg_code=layer.GetSpatialRef().GetAuthorityCode(None)
shape=layer.GetFeature(0).ExportToJson(as_object=True)['geometry']
shape["properties"]={"name": "EPSG:{0}".format(epsg_code)}#latlon projection            

query={
  "type": "CropRaster",
  "params": {
    "properties": {
      "outputFileName": outputFileName,
      "cutline": True,
      "tiled": True,
      "compressed": True,
      "overviews": True
    },
    "cube": {
      "code": cube_code,
      "workspaceCode": workspace,
      "language": "en"
    },
    "dimensions": [
      {
        "code": "SEASON",
        "values": [
          season
        ]
      },
      {
        "code": "STAGE",
        "values": [
          stage
        ]
      },
      {
        "code": "YEAR",
        "values": [
          year
        ]
      }
    ],
    "measures": [
      measure
    ],
    "shape": shape
  }
}


Poster la charge de la requête (QueryPayload) avec une clé d’accès dans l’entête (Header). En réponses, obtient un url pour requérir le travail

In [6]:
resp_query=requests.post(path_query,headers={'Authorization':'Bearer {0}'.format(AccessToken)},
                                          json=query)
resp_query = resp_query.json()
job_url=resp_query['response']['links'][0]['href']

job_url

'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/3b623bf6-33ea-4d31-92fc-1a69a800ca3a'

# Étape 4: Obtenir les résultats du travail (Job)


Cela mettra du temps avant que le travail ne soit fini. Une fois le travail fini, son statut changera de « En cours » (RUNNING) à « Terminé » (COMPLETED) ou « terminé avec erreurs » (COMPLETED WITH ERRORS). Si c’est terminé, les résultats de la série chronologique de la zone peuvent être trouvés à partir de Réponse « sortie ».

In [7]:
i=0
print('RUNNING',end=" ")
while i==0:        
    resp = requests.get(job_url)
    resp=resp.json()
    
    if resp['response']['status']=='RUNNING':
        print('.',end =" ")
    if resp['response']['status']=='COMPLETED':
        results=resp['response']['output']
        print(resp['response']['output'])
        i=1
    if resp['response']['status']=='COMPLETED WITH ERRORS':
        print(resp['response']['log'])
        i=1       

RUNNING . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . {'downloadUrl': 'https://io.apps.fao.org/gismgr/download/a0d995ce-91d3-4aca-8b18-4386ce4bc8c1/L2_PHE_17s1_s_clipped.tif'}
